In [1]:
import pandas as pd
import os

# Importando tabelas

In [2]:
df_dados = pd.read_excel('dados.xlsx')
df_fator = pd.read_excel('fatores.xlsx')

lst_meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
lst_meses_dados = ['Dez anterior','Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov']
lst_layout = ['Galpões', 'Complemento','Registro', 'Empresa', 'Tipo de imóvel']

# Gerando Data Frame das diferenças

In [3]:
df_diferença = df_dados[lst_layout]

for m, d in zip(lst_meses, lst_meses_dados):
    dif = df_dados.loc[:, m] - df_dados.loc[:, d]
    df_diferença.loc[:, m] = dif


df_diferença = df_diferença.loc[~df_diferença['Jan'].isna(), :].reset_index(drop=True)

df_fator.loc[3, 'Volume max de água m3'] = 999999


C:\Users\Gabriel\AppData\Local\Temp\ipykernel_14728\2604391656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_14728\2604391656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_14728\2604391656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

# Gerando Data Frame dos preços

In [4]:
# As funções definidas a seguir foram somente para organizar a funcionalidade do loop

# def df_taxas_xm(index, m, type):
#     df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
#     return df_temp

# def df_taxas_Mxm(index, m, type):
#     #type foi utilizada fora da função como o tipo de imóvel. No caso do loop, como var
#     df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]
#     return df_temp

# def df_taxas_Mx(index, m, type):
#     df_temp = df_preço.loc[(df_diferença[m] >= df_fator.loc[index,'Volume min de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
#     return df_temp


df_preço = df_diferença.loc[df_diferença['Galpões'] != 'Poço', lst_layout]

for m in lst_meses:
    
    for type in ['Industrial', 'Residencial']:
        df_preço.loc[(df_diferença[m] < df_fator.loc[0,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m] = df_fator.loc[0, f'Fator {type}']
        
        for index in range(1, 4):
            df_preço_temp = df_diferença.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]  
            df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m] = df_preço_temp * df_fator.loc[index, f'Fator {type}']

df_preço = df_preço.round(2)

# TOTAL
num_index = df_preço.index[-1]
df_preço.loc[num_index + 1,:] = df_preço.sum(numeric_only=True, axis=0)
df_preço.loc[num_index + 1, 'Galpões'] = 'TOTAL'

df_preço.to_excel('Tabela preços água.xlsx', index=False)

# # TOTAL
# last_index = df_preço.index[-1]
# new_row = df_preço.sum(numeric_only=True, axis=0)
# new_row['Galpões'] = 'TOTAL'
# df_preço = pd.concat([df_preço, new_row]).reset_index(drop=True)



        


# Separando arquivos na pasta compartilhamento

In [5]:
if not os.path.exists(r'compartilhamento'):
    os.mkdir(r'compartilhamento')


lst_empresa = df_preço['Empresa'].value_counts().index.tolist()

for comp in lst_empresa:

    # Criação de DataFrame para cada empresa
    df_preço_temp = df_preço.loc[df_preço['Empresa'] == comp, ['Galpões','Complemento','Registro']+lst_meses]
    df_preço_temp = df_preço_temp.dropna(how='all', axis=1).reset_index(drop=True)
    display(df_preço_temp)
    
    # lista com meses completos
    lst_meses_uteis = df_preço_temp.columns.tolist()
    lst_meses_uteis.remove('Galpões')
    try:
        lst_meses_uteis.remove('Complemento')
    except:
        continue
    lst_meses_uteis.remove('Registro')

    # implementação de TOTAL
    
    
    num_index = df_preço_temp['Registro'].value_counts().sum()
    df_preço_temp.loc[num_index,:] = df_preço_temp.sum(numeric_only=True, axis=0)
    df_preço_temp.loc[num_index, 'Galpões'] = 'TOTAL'
    
    
    
    # Formatação de R$
    df_preço_temp[lst_meses_uteis] = df_preço_temp[lst_meses_uteis].map(lambda x: f'R$ {x}')
    
    
    df_preço_temp.to_excel(fr'compartilhamento\{comp}.xlsx', index=False)
    

,Galpões,Complemento,Registro,Jan,Fev,Mar,Abr,Mai,Jun
0,Galpão 4,NaN,A99L488887,413.43,460.28,406.11,660.10,629.73,419.58
1,Galpão 10,NaN,A13L080158,240.58,260.98,104.18,203.98,196.66,246.93
2,Galpão 11,R. de Incêncio,A13L080153,51.04,51.04,51.04,51.04,51.04,51.04


,Galpões,Registro,Jan,Fev,Mar,Abr,Mai,Jun
0,Galpão 1,A79B53393,1212.47,773.55,857.21,616.55,859.5,786.16


,Galpões,Registro,Jan,Fev,Mar,Abr,Mai,Jun
0,Galpão 2,A99L488883,370.0,361.51,437.44,420.36,426.61,467.21


,Galpões,Complemento,Registro,Jan,Fev,Mar,Abr,Mai,Jun
0,Galpão 8,Piso Inferior,A09S372870,51.04,51.04,51.04,51.04,51.04,51.04
